# Aula 03 - Maratona Python com BI

In [1]:
#https://www.youtube.com/watch?v=kuF94ZmZO8s&t=125s

## 0. Review
- Aprender funcoes personalizadas
- Tratar e se precaver dos erros
- quebrar o texto com BS4
- usar o loop and while

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

In [156]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
#criando funcao para pegar aquivo de html local
def PegaHtm1Local(nome, path_subpasta = ''):

    # trata path do sublocat
    if path_subpasta == '':
        path_subpasta = ''
    else :
        path_subpasta = path_subpasta + '/'
        
    # abra o arquivo, e alimente variavel.
    # o 'r' significa que é somente para Ler o arquivo.
    with open(path_subpasta + str(nome) + '.html', 'r') as f:
        soup = BeautifulSoup(f, 'html5lib')
    
    return soup

## 1. Usando APIs

In [7]:
# Se tiver API, como usar
import requests
import ast
import pandas as pd

In [11]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
headers = { 'User-Agent' : user_agent}
url = "http://api.willcode.tech/funcionarios/?USUARIO=USUARIO&SENHA=SENHA&ACAO=LISTAR-TODOS&SHOWEMPTY"
response = requests.request("GET",url)


In [14]:
response.json()
pd.DataFrame.from_dict(response.json())

{'API': '/api/funcionarios/',
 'ERROR_TYPE': 'AuthException',
 'ERROR_CODE': '',
 'ERROR_MESSAGE': 'Falha ao realizar login. Verifique suas credenciais',
 'REQUEST': '{"USUARIO":"USUARIO","SENHA":"SENHA","ACAO":"LISTAR-TODOS","SHOWEMPTY":""}'}

## 2. Alimentar Banco de Dados

In [15]:
#Importanto bibliotecas
import time
import pandas as pd

from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import bs4

In [ ]:
#Definindo o header de consulta na web
user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
headers = {'User-Agent' : user_agent}

In [76]:
# Funcao para chamar a pagina WEB
def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()
    
    except:
        pass
    

In [77]:
# Funcäo para pegar a pägina encontrada e transformar em um objeto utilizävel
def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html,'html.parser')
    return soup


In [78]:
# Funcao para capturar as informacoes da "tabelinha cabecalho" da pagina
def Cabecalho(html):
    dt = html.find_all('dt') 
    dd = html.find_all('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text()
        dic[x]=y
    return dic

In [199]:
#Funcao para capturar o conteudo da pagina e transformar em dicionario
def Conteudo(proposicao,ano):
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url)
    dic = Cabecalho(html)
    dic['Proposicao'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text()
    #Ajustando a data no formato AAAAMMDD
    data_reuniao = str(dic["Reunião"]).split('/')  
    dic.update({'Reunião':data_reuniao[2]+data_reuniao[1]+data_reuniao[0]})
    data_deliberacao = str(dic["Deliberação"]).split('/')  
    dic.update({'Deliberação':data_deliberacao[2]+data_deliberacao[1]+data_deliberacao[0]})
    return dic

In [200]:
teste2 = Conteudo(proposicao,ano)
teste2

{'Reunião': '20210211',
 'Deliberação': '20210211',
 'Situação': 'Proposição Aprovada',
 'Assunto': 'Manutenção de via pública',
 'Autor': 'Vereador Valentim Blasius.',
 'Proposicao': 200,
 'Ano': 2021,
 'Texto': 'O Vereador abaixo firmado requer, após ouvido o Plenário, encaminhamento de cópia da presente Indicação à Secretaria de Urbanização e Meio Ambiente solicitando manutenção de calçada da rua 25 de Maio, defronte ao nº 266, Carijós. A pedido de moradores.'}

In [80]:
#Funcao Geral que retorna um df com todas as informacoes e controla erros
def TabelaResultados(iniciar_em, quantidade, ano, erros_admissiveis, segundos_espera):
    
    ultima_consulta = iniciar_em + quantidade -1

    #Erros
    erros = 0

    #Variaveis para o loop
    lista = []
    i=1

    while iniciar_em <= ultima_consulta and erros <= erros_admissiveis:
        try:
            x = Conteudo(iniciar_em,ano)
            lista = lista + [Conteudo(iniciar_em,ano)]
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        #incrementos
        iniciar_em += 1
        i += 1

    return pd.DataFrame(lista)

In [201]:
teste = TabelaResultados(500, 5, 2021, 2, 0.5)
teste

,Reunião,Deliberação,Situação,Assunto,Autor,Proposicao,Ano,Texto
0,20210408,20210408,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,500,2021,"O vereador que está subscreve, no uso das atri..."
1,20210408,20210408,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Flávio Augusto Ferri Molinari.,501,2021,"O vereador que está subscreve, no uso das atri..."
2,20210408,20210408,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,502,2021,"A vereadora que esta subscreve, no uso das atr..."
3,20210408,20210408,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,503,2021,"A vereadora que esta subscreve, no uso das atr..."
4,20210408,20210408,Proposição Aprovada,Trânsito,Vereador Márcio Moisés Selhorst.,504,2021,"O vereador que esta subscreve, no uso das atri..."


## 2. Criando o Banco de Dados e Tabelas

### 2.1 Youtube Video pra instalar o SSMS e SQL Server

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('NthkdjtEKvc', width=720, height=405)

### 2.2 Criando o BD

In [ ]:
list(teste.columns)

In [ ]:
teste.dtypes

>

> Criando um script para criar a table Usando tag sql para formato

```sql
CREATE TABLE Proposicoes(
    DataReuniao DATE,
    DataDeliberacao DATE,
    Situacao VARCHAR(200)
    Assunto VARCHAR(1000)
    Autor VARCHAR(1000)
    Proposicao INT,
    Ano INT,
    Texto VARCHAR(MAX)
)
```
> Usando tag sql para formato

## 2.3 Alimentando as Tables

In [94]:
# Importando a bib que conecta com o Banco
import pyodbc
pyodbc.version

'4.0.35'

In [202]:
'''
# com senha
conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'Indaial',
                      UID = 'sa',
                      PWD = 'senha')
'''
# Com autenticacao Windows
conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'Indaial')
query = '''
    Select 
    * 
    from Proposicoes
'''

sql_query = pd.read_sql_query(query,conn)
sql_query

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [203]:
# Crinado uma base df com as colunas
base = pd.DataFrame(columns=['Reunião','Deliberação','Situação','Assunto','Autor','Proposicao','Ano','Texto'])
base

,Reunião,Deliberação,Situação,Assunto,Autor,Proposicao,Ano,Texto


In [204]:
teste

,Reunião,Deliberação,Situação,Assunto,Autor,Proposicao,Ano,Texto
0,20210408,20210408,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,500,2021,"O vereador que está subscreve, no uso das atri..."
1,20210408,20210408,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Flávio Augusto Ferri Molinari.,501,2021,"O vereador que está subscreve, no uso das atri..."
2,20210408,20210408,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,502,2021,"A vereadora que esta subscreve, no uso das atr..."
3,20210408,20210408,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,503,2021,"A vereadora que esta subscreve, no uso das atr..."
4,20210408,20210408,Proposição Aprovada,Trânsito,Vereador Márcio Moisés Selhorst.,504,2021,"O vereador que esta subscreve, no uso das atri..."


In [205]:
# Colocando tudo em uma unica BLOCO
base = pd.DataFrame(columns=['Reunião','Deliberação','Situação','Assunto','Autor','Proposicao','Ano','Texto'])
teste = base.append(teste).fillna('')

conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'Indaial')

cursor = conn.cursor()

for index, row in teste.iterrows():
    
    cursor.execute('''
    
        INSERT INTO Proposicoes (
            DataReuniao,
            DataDeliberacao,
            Situacao,
            Assunto,
            Autor,
            Proposicao,
            Ano,
            Texto
                      
        )
        Values(?,?,?,?,?,?,?,?)''', # mesma qtde de variaveis
        
        row['Reunião'],
        row['Deliberação'],
        row['Situação'],
        row['Assunto'],
        row['Autor'],
        row['Proposicao'],
        row['Ano'],
        row['Texto']
    )

conn.commit()
cursor.close()

In [206]:
# Criando uma function para retorno da query
def SQLSelect(query):
    conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'Indaial')
    out = pd.read_sql_query(query,conn)
    return out

In [258]:
query_all = '''
    Select 
    * 
    from Proposicoes
    order by Ano, Ano desc, Proposicao desc
'''
query_max = '''
    SELECT
        maior = max(Proposicao)
    from Proposicoes
'''

SQLSelect(query_all)


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [208]:
# Criando uma function para ISERT da query
def SQLInsertProposicoes(TabelaProposicoes):
    base = pd.DataFrame(columns=['Reunião','Deliberação','Situação','Assunto','Autor','Proposicao','Ano','Texto'])
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'Indaial')

    cursor = conn.cursor()

    for index, row in TabelaProposicoes.iterrows():

        cursor.execute('''

            INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto

            )
            Values(?,?,?,?,?,?,?,?)''', # mesma qtde de variaveis

            row['Reunião'],
            row['Deliberação'],
            row['Situação'],
            row['Assunto'],
            row['Autor'],
            row['Proposicao'],
            row['Ano'],
            row['Texto']
        )

    conn.commit()
    cursor.close()
    

In [209]:
SQLInsertProposicoes(teste)

In [158]:
# Function to truncate a table
def SQLTruncate(NomeTabela):
    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'Indaial')

    cursor = conn.cursor()

    cursor.execute(f'''
    
                   TRUNCATE TABLE {NomeTabela}
                     
                   ''')
    conn.commit()
    cursor.close()

In [210]:
SQLTruncate('Proposicoes')

In [211]:
SQLSelect(query_all)

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [212]:
SQLSelect(query_max)

,maior
0,None


## 3. Alimentacao Incremental

### 3.1 Modificando a funcao de Busca

In [215]:
# Limpando os dados so para demonstracao
SQLTruncate('Proposicoes')

# ============ INICIO ===================
proposicao = 200
ano = 2021
dados = Conteudo(proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ============  FIM  ===================

# Vizualizando os dados
SQLSelect(f'SELECT * FROM Proposicoes where Proposicao = {proposicao} and Ano = {ano}')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2021-02-11,2021-02-11,Proposição Aprovada,Manutenção de via pública,Vereador Valentim Blasius.,200,2021,"O Vereador abaixo firmado requer, após ouvido ..."


In [214]:
SQLSelect(query_all)

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2021-02-11,2021-02-11,Proposição Aprovada,Manutenção de via pública,Vereador Valentim Blasius.,200,2021,"O Vereador abaixo firmado requer, após ouvido ..."


In [223]:
ano = 2021
dados = SQLSelect(f'select Proposicao = max(proposicao) from Proposicoes where Ano = {ano}')
dados['Proposicao'].loc[0]

215

In [234]:
# ============ INICIO ===================
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(proposicao) from Proposicoes where Ano = {ano}')
proxima_proposicao = int(dados_ano['Proposicao'].loc[0])+1

dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
SQLSelect(query_all)
# ============  FIM  ===================

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [242]:
# Limpando os dados so para demonstracao
SQLTruncate('Proposicoes')
SQLSelect(query_all)

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [237]:
# ============ INICIO ===================
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]

if ultima_proposicao == None:
    proxima_proposicao = 1
else:
    proxima_proposicao = int(ultima_proposicao)+1

dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
SQLSelect(query_all)
# ============  FIM  ===================

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2021-02-01,2021-02-01,Proposição Aprovada,Trânsito,Vereador Remir José de Faveri.,1,2021,"O vereador que esta subscreve, no uso das atri..."
1,2021-02-01,2021-02-01,Proposição Aprovada,Manutenção de via pública,Vereador Remir José de Faveri.,2,2021,"O vereador abaixo firmado requer, após ouvido ..."
2,2021-02-01,2021-02-01,Proposição Aprovada,Manutenção de via pública,Vereador Valentim Blasius.,3,2021,"O vereador abaixo firmado requer, após ouvido ..."


In [239]:
def InsereProximaProposicao(ano):
    dados_ano = SQLSelect(f'select Proposicao = max(proposicao) from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]

    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao)+1

    dados = Conteudo(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsertProposicoes(tabela)

In [243]:
InsereProximaProposicao(2020)
SQLSelect(query_all)

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2020-02-03,2020-02-03,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Osvaldo Metzner.,1,2020,"O vereador que esta subscreve, no uso das atri..."


In [248]:
def BuscaGravaDadosAno(ano, quantidade = 99999, erros_admissiveis = 2, segundos_espera = 0.5):
    
    #erros
    erros = 0
    
    #variaveis para o loop
    i = 1
    lista = []
    
    while erros <= erros_admissiveis:
        
        try:
            InsereProximaProposicao(ano)
        except:
            erros += 1
            pass
        
        time.sleep(segundos_espera)
        
        i += 1

In [ ]:
# Limpando os dados so para demonstracao
SQLTruncate('Proposicoes')
SQLSelect(query_all)

In [249]:
ano=1996
BuscaGravaDadosAno(ano)

In [252]:
# Limpando os dados so para demonstracao
SQLTruncate('Proposicoes')
SQLSelect(query_all)

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [259]:
ano_inicial = 1996
ano_final = 1997

for i in list(range(ano_inicial, ano_final+1)):
    print('Iniciando gravacao dos dados do ano: ',i)
    try:
        BuscaGravaDadosAno(i, segundos_espera=0.3)
    except:
        pass
print("Insercao finalizada")

Iniciando gravacao dos dados do ano:  1996
Iniciando gravacao dos dados do ano:  1997
Insercao finalizada


In [261]:
query_all = '''
    Select 
    * 
    from Proposicoes
    order by Ano desc, Proposicao desc
'''
query_max = '''
    SELECT
        maior = max(Proposicao)
    from Proposicoes
'''

SQLSelect(query_all)

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1997-02-27,1997-02-27,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Alinor Lauth.,232,1997,"Limpeza, patrolamento e macadamização da rua T..."
1,1997-02-27,1997-02-27,Proposição Aprovada,Diversos,Vereador Sergio Almir dos Santos.,231,1997,Reformar as dependências da Associação de Mora...
2,1997-02-27,1997-02-27,Proposição Aprovada,Irrigação,Vereador Sergio Almir dos Santos.,230,1997,"Irrigar a rua 16 de Julho, bairro Carijós. ..."
3,1997-02-27,1997-02-27,Proposição Aprovada,Irrigação,Vereador Sergio Almir dos Santos.,229,1997,"Irrigar a rua 25 de Julho, bairro Carijós. ..."
4,1997-02-27,1997-02-27,Proposição Aprovada,Irrigação,Vereador Sergio Almir dos Santos.,228,1997,"Irrigar a rua 11 de Junho, bairro Carijós. ..."
...,...,...,...,...,...,...,...,...
251,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,5,1996,Prolongamento de rede d'água na Rua Lorenz até...
252,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,4,1996,prolongamento da rede d'água na Rua Reinhold S...
253,1996-02-22,1996-02-22,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Henrique Fritz.,3,1996,alargamento da Rua ID 90 ...
254,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,2,1996,construção de Escola nas imediações dos Loteam...


In [262]:
# https://www.youtube.com/watch?v=kVYpd_CWLEM&list=PLQpSyz5rZmJr3OLXzK9DDwLU14VOpAbXh&index=4&t=3s